In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import os, sys

def get_dir_n_levels_up(path, n):
    # Go up n levels from the given path
    for _ in range(n):
        path = os.path.dirname(path)
    return path


proj_root = get_dir_n_levels_up(os.path.abspath("__file__"), 2)
sys.path.append(proj_root)

from pathlib import Path
import traceback
from typing import Dict
import logging

from liftoff import parse_opts

from opinion_dqn import AgentDQN
from utils import my_logging
from utils.experiment import seed_everything, create_path_to_experiment_folder, build_environment
from utils.generic import convert_namespace_to_dict
import yaml

In [3]:
# softmax in last layer because it scales with N instead of 2^N where N is the nr of agents

# might need to do policy gradient

# next step: Q iteration with action representation

In [4]:
train_env = build_environment()
train_env.action_space.shape[0]

4

In [5]:
experiment_yaml = "2025Jan15-202913_configs"
yaml_path = Path(
    r"D:\Work\repos\RL\phd-rl-algos\dqn\opinion_dynamics\experiments\results"
) / experiment_yaml / "0000_estimator.args_.lin_hidden_out_size_32" / "0" / "cfg.yaml"


with open(yaml_path, 'r') as file:
    config = yaml.safe_load(file)
seed = int(os.path.basename(config["out_dir"]))

seed_everything(seed)

logs_file = os.path.join(config["out_dir"], "experiment_log.log")

logger = my_logging.setup_logger(
    name=config["experiment"],
    # log_file=logs_file,
    level=logging.INFO,
)

logger.info(f"Starting experiment: {config['full_title']}")

### Setup environments ###
train_env = build_environment()
validation_env = build_environment()

### Setup output and loading paths ###

path_previous_experiments_outputs = None
if "restart_training_timestamp" in config:
    path_previous_experiments_outputs = create_path_to_experiment_folder(
        config,
        config["out_dir"],
        config["restart_training_timestamp"],
    )

experiment_agent = AgentDQN(
    train_env=train_env,
    validation_env=validation_env,
    experiment_output_folder=config["out_dir"],
    experiment_name=config["experiment"],
    resume_training_path=path_previous_experiments_outputs,
    save_checkpoints=True,
    logger=logger,
    config=config
)

logger.info(
    f'Initialized agent with models: {experiment_agent.policy_model}'
)

experiment_agent.train(train_epochs=config["epochs_to_train"])

logger.info(
    f'Finished training experiment: {config["full_title"]}, seed: {config["seed"]}'
)

my_logging.cleanup_file_handlers(experiment_logger=logger)


2025-01-29 23:37:00,591 - opinion_agent_dqn - INFO - Starting experiment: 2025Jan15-202913_configs_estimator.args_.lin_hidden_out_size=32
2025-01-29 23:37:00,592 - opinion_agent_dqn - INFO - Loaded configuration settings.
2025-01-29 23:37:01,395 - opinion_agent_dqn - INFO - Initialized newtworks and optimizer.
2025-01-29 23:37:01,396 - opinion_agent_dqn - INFO - Initialized agent with models: OpinionNet(
  (fc): Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
  )
  (predict_A_b_q): Linear(in_features=32, out_features=18, bias=True)
)
2025-01-29 23:37:01,396 - opinion_agent_dqn - INFO - Starting training session at: 0
2025-01-29 23:37:01,397 - opinion_agent_dqn - INFO - Starting training epoch at t = 0
2025-01-29 23:37:01,887 - opinion_agent_dqn - INFO - States shape: torch.Size([32, 1, 4]), Next States shape: torch.Size([32, 1, 4])
2025-01-29 23:37:01,888 - opinion_agent_d

d:\Work\repos\RL\phd-rl-algos\dqn\opinion_dynamics\replay_buffer.py:57: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  actions = torch.tensor(actions, dtype=torch.float32)


RuntimeError: Index tensor must have the same number of dimensions as input tensor